# Artificial Neural Network

In [2]:
import time                     #time
import numpy as np              #tools for computing array
import pandas as pd             #data manipulation
import matplotlib.pyplot as plt #plot or graphic
from sklearn.metrics import mean_squared_error #fungsi untuk menghitung mse rmse

## *Fungsi ANN
Fungsi ANN akan dibangun dengan menggunakan referensi https://www.coursera.org/learn/neural-networks-deep-learning/lecture/znwiG/forward-and-backward-propagation

Tahapan ANN dan beberapa fungsi tambahan dalam file ini dapat dilihat pada link berikut:
- <a href='#initparam'>Inisialisasi parameter</a>
- <a href='#fa'>Activation function</a>
- <a href='#forward'>Forward propagation</a>
- <a href='#cost'>Compute cost</a>
- <a href='#backward'>Backward propagation</a>
- <a href='#update'>Update parameters</a>
- <a href='#ann'>Modelling ANN</a>
- <a href='#predict'>Predict function</a>
- <a href='#plot'>Plot function</a>

Langkah pertama dalam ANN adalah inisialisasi parameter. Initialize_params merupakan fungsi untuk menginisialisasi parameter bobot dan bias sesuai dimensi layer dan neuron yang ditentukan.

<a id='initparam'></a>

In [27]:
def initialize_params(layer_dims, jml_input, random_state=1):
    '''
    Inputs:
    layer_dims -- size of hidden layer
    
    Returns:
    params -- dict of param (weight n bias)
        Wi.shape = (n_i+1, n_i)
        bi.shape = (n_i+1,1)
    '''    
    # nambah input dan output layer
    layer_dims.insert(0,jml_input) #input
    layer_dims.insert(len(layer_dims), 1) #out
    
    # init
    if random_state: np.random.seed(random_state)
    params = {}
    n = len(layer_dims)
    
    for i in range(n-1):
        # weight param
        params['W'+str(i+1)] = np.random.randn(layer_dims[i+1], layer_dims[i])*0.1
        # bias param
        params['b'+str(i+1)] = np.zeros((layer_dims[i+1],1))
        
    return params

Fungsi aktivasi akan dibangun dalam fungsi fa, dan akan dilengkapi dengan turunan fungsi aktivasi masing-masing. Fungsi aktivasi nantinya akan digunakan pada tahap forward propagation, sedangkan turunan fungsinya akan digunakan pada tahap backward propagation. Fungsi aktivasi yang dimaksud yaitu:
- Tanh
- Sigmoid
- ReLU
- Leaky ReLU

<a id='fa'></a>

In [4]:
def fa(x, activation_func, derivative=False, alpha=0.01):
    '''
    Activation functions 
    Input:
    x -- input for the function
    activation_func -- type of func: tanh, sigmoid, relu, leaky relu
    derivative -- if True, then x will calc with derivative func
    alpha -- only used when activation func is 'leaky relu'
    
    Return:
    y -- output of the func
    '''
    # ident
    if activation_func == 'identity':
        y = x if not derivative else 1
    
    # tanh
    elif activation_func == 'tanh':
        y = np.tanh(x) if not derivative else 1-np.power(np.tanh(x),2)
    
    # sigmoid
    elif activation_func == 'sigmoid': 
        a = np.exp(-x)
        sig = 1/(1+a)
        y = sig if not derivative else sig*(1-sig)
        
    # relu
    elif activation_func == 'relu':
        y = np.maximum(0,x) if not derivative else np.where(x<0, 0, 1)
            
    # leaky relu
    elif activation_func == 'leaky_relu':
        y = np.maximum(alpha*x,x) if not derivative else np.where(x<0, alpha, 1)        
    
    # error karna selain fungsi di atas    
    else: 
        raise Exception (f' \'{activation_func}\' function not found')
        
    return y

Tahap selanjutnya adalah forward propagation. Data input akan dikalikan dengan parameter bobot dan ditambah bias. Selanjutnya hasil tersebut akan diaktivasi dengan fungsi aktivasi. Proses ini terus berlanjut hingga mencapai layer output.

Forward Propagation untuk layer $l$ dapat dituliskan:
$$Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$$
$$A^{[l] } = \mbox{fa}^{[l]}(Z^{[l]})$$
$$\hat{Y} = A^{[n]} = \mbox{fa}^{[n]}(Z^{ [n]})$$

dimana Z adalah perkalian bobot dengan input/aktivasi layer sebelumnya ditambah bias; b adalah bias; A adalah output fungsi aktivasi (fa) dari Z; 

<a id='forward'></a>

In [5]:
def forward_propagation(X,params, activation_func):
    '''
    Inputs:
    X -- data X
    params -- dict of params (from initialize_params)
    activation_func -- activation function for hidden layer, 
                        output layer use identity (sementara)
    
    Returns:
    AL -- output of last activation (in output layer)
    cache -- dict containing Z, A
    '''
    cache = {}
    cache['A0'] = X
    
    # calc Zi and Ai
    # untuk setiap layer hidden n output
    for i in range(round(len(params)/2)):
        # Zi
        cache['Z'+str(i+1)] = np.dot(params['W'+str(i+1)],cache['A'+str(i)]) +\
        params['b'+str(i+1)]
        
        # activation Zi -> Ai
        cache['A'+str(i+1)] = fa(cache['Z'+str(i+1)], activation_func)
    
    # untuk output aj -> activation function pake identity
    cache['A'+str(i+1)] = cache['Z'+str(i+1)]
    #del cache['A0']
    
    return cache['A'+str(i+1)], cache #AL -> output prediksi

Hasil forward propagation akan dihitung cost/loss dengan menghitung selisih nilai prediksi dan nilai sebenarnya.
Cost $J$ pada $y^{(i)}$ tersebut dapat dihitung dengan: 
$$J = - \frac{1}{m} \sum\limits_{i = 0}^{m} \large\left(\small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right)  \large  \right) \small $$
atau dengan SSE
$$J = \frac{1}{2} \sum\limits_{i}\small\left( y^{(i)}-\mbox{fa} (z^{(i)}) \small\right)^{2}$$
Atau bisa dengan menggunakan RMSE.
Chunk di bawah menggunakan SSE untuk menghitung cost dari forward propagation.

<a id='cost'></a>

In [6]:
def compute_cost(AL, Y):
    '''
    Input:
    AL -- output of last activation / prediction 
    Y -- true label / target data
    
    Return:
    cost -- cross entropy cost 
    '''
    m = Y.shape[1]
    Y = Y.T.reshape(m,1)
    AL = AL.T.reshape(m,1)
    
    # Calculate cost
    #logprobs = np.multiply(np.log(AL), Y) + np.multiply(np.log(1-AL), 1-Y)
    #cost = - np.sum(logprobs)/m
    #cost = mean_squared_error(Y, AL, squared=False) #RMSE if squared is false
    #cost = np.sqrt(np.average((Y-AL) ** 2, axis=0)) #RMSE
    cost = np.average((Y-AL) ** 2, axis=0) / 2 #MSE
    
    # Clean
    cost = float(np.squeeze(cost))
    
    return cost

Tahap selanjutnya adalah backward propagation. Error yang dihitung berdasarkan nilai target dan hasil prediksi akan dipropagasikan ke belakang dengan mengubah bobot dan biasnya. Backward propagation untuk layer $l$ dapat dihitung menggunakan rumus:
$$dZ^{[l]} =  dA^{[l]}*\mbox{fa}^{[l]'}(Z^{[l]})$$ 
$$dW^{[l]} = \frac{1}{m}dZ^{[l]} . A^{[l-1]T}$$
$$db^{[l]} = \frac{1}{m}\mbox{np.sum}(dZ^{[l]} \mbox{, axis=1, keepdims=True})$$
$$dA^{[l-1]} = W^{[l]T}.dZ^{[l]}$$
Setelah mendapat hasil backward propagation, hasil tersebut digunakan untuk mengupdate parameter bobot dan bias.

<a id='backward'></a>

In [7]:
def backward_propagation(params, cache, Y, activation_func):
    """
    Input:
    params -- dict containing params
    cache -- dict containing Z and A
    Y -- true labels
    activation_func -- the activation func like in forward prop
    
    Return:
    grads -- dict containing gradients with respect to different params
    """
    m = Y.shape[1]
    grads, temp = {},{}
    l = round(len(params)/2)
    
    # Backward propagation: calc dWi, dbi 
    # utk setiap layer tapi dibalik
    for i in reversed( range(1,l+1) ):
        # dZ / delta error
        # output
        if (i == l):
            ## dz = da * g'(z); da = dL/da
            selisih = Y - cache['A'+str(i)]
            temp['dZ'+str(i)] = -selisih # selisih / np.sqrt(
                #np.average((Y-cache['A'+str(i)]) ** 2, axis=0))
        # hidden
        else:
            temp['dZ'+str(i)] = \
                np.dot(params['W'+str(i+1)].T, temp['dZ'+str(i+1)])* \
                fa(cache['Z'+str(i)],activation_func,True)#(1 - np.power(A1, 2))
        
        # dW db
        grads['dW'+str(i)] = np.dot(temp['dZ'+str(i)], cache['A'+str(i-1)].T)/m
        grads['db'+str(i)] = np.sum(temp['dZ'+str(i)], axis=1, keepdims=True)/m
    
    return grads

<a id='update'></a>

In [8]:
def update_params(params, grads, delta_prev, momentum, learning_rate):
    '''
    Input: 
    params -- dict of params
    grads -- dict of gradients
    delta_prev -- previous dict delta params for calculate momentum
    momentum -- momentum rate param
    learning_rate -- learning_rate param
    
    Return:
    params -- updated params
    delta_prev -- previous dict delta params updated
    '''
    # init
    l = round(len(params)/2)
    
    for i in reversed( range(1,l+1) ):    
        # previous delta params
        ## jika delta_prev masi kosong, init delta_prev dengan 0 sesuai dimensi params
        if not bool(delta_prev): 
            delta_prev['W'+str(i)] = params['W'+str(i)]*0
            delta_prev['b'+str(i)] = params['b'+str(i)]*0
        # update delta_prev
        delta_prev['W'+str(i)] = learning_rate*grads['dW'+str(i)]
        delta_prev['b'+str(i)] = learning_rate*grads['db'+str(i)]
        # update params
        params['W'+str(i)] -= momentum * delta_prev['W'+str(i)]
        params['b'+str(i)] -= momentum * delta_prev['b'+str(i)] 
        
    return params, delta_prev

Fungsi-fungsi di atas dijadikan satu fungsi sehingga membentuk suatu fungsi pemodelan ANN. Fungsi tersebut membutuhkan beberapa input seperti:
- X,Y : data input dan targetnya
- layer_dims : berisi jumlah hidden layer beserta jumlah neuronnya dalam bentuk list
- learning_rate : parameter learning rate untuk mengupdate gradient descent. Nilai default akan diset dengan nilai 0.001
- epoch : banyaknya iterasi pembelajaran. Nilai default akan diset sebanyak 1000 pengulangan
- print_cost : apakah fungsi ANN perlu menampilkan hasil cost atau tidak, jika ya, maka akan menampilkan cost setiap 100 atau 10 perulangan

<a id='ann'></a>

In [9]:
def nn_model(X, Y, layer_dims, activation_func, 
             momentum=0.9,learning_rate=0.001, epoch=1000,
             print_cost=False, random_state=1, early_stop=False, param=None):
    '''
    Inputs:
    X,Y -- data
    layer_dims -- list containing the input size n each layer size
    momentum -- momentum rate for gradient descent update
    learning_rate -- learning rate of gradient descent update rule
    epoch -- num of loop/num iteration
    print_cost -- if True, it prints the cost every 100 steps
    random_state -- random seed 
    early_stop -- jika True, maka loop akan dihentikan bila gada perbedaan cost
                sebelumnya dengan cost selanjutnya
    (param -- param weight dan bias)###########################################
    
    Returns:
    params -- params learnt by model
    AL -- prediction result
    cost -- nilai cost / loss yang dihasilkan; selisih pred dan true
    '''
    # init n sesuaiin x,y
    X = X.T
    Y = Y.T.reshape(1,Y.shape[0])
    cost = np.zeros(Y.shape)
    costs = []
    delta_prev={}
    
    # init param weight n bias
    # jika gada param weight dan bias yang dimasukin, berarti random aja
    if param is None:
        params = initialize_params(layer_dims, jml_input=X.shape[0], 
                                   random_state=random_state)
    else:
        raise Exception('Belum di set wkwkwk; Tar akan disesuaiin')
    
    # loop (gradient descent)
    for i in range(0, epoch):
        # forward
        AL, caches = forward_propagation(X, params, activation_func)
        # cost
        ## jika dikasi early stop
        if early_stop:
            ## hitung selisih nilai cost min sebelumnya dg sekarang
            dcost_min = np.min(np.abs(cost-compute_cost(AL,Y)))
            ## jika kurang dari threshold maka hentikan
            if dcost_min <= 1e-4 :
                if print_cost:
                    print(dcost_min)
                    print(f'Iterasi dihentikan pada {i} dari {epoch} karena tidak ada perubahan cost')
                break
        cost = compute_cost(AL,Y)
        # back
        grads = backward_propagation(params, caches, Y, activation_func)
        # update params
        params, delta_prev = update_params(params, grads, delta_prev, momentum, learning_rate)
        
        # Print the cost every 100 or 10 training example
        bil = 10 if epoch<=100 else 100
        if print_cost and i % bil == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % bil == 0:
            costs.append(cost)
            
    # plot the cost
    if print_cost:
        plot([costs], xlabel='iterations (per hundreds)', ylabel='cost',
             title="Cost Graph with learning rate =" + str(learning_rate))
        
    t = 1000 * time.time() # current time in milliseconds
    np.random.seed(int(t) % 2**32)
    
    return params, np.squeeze(AL.T), cost

Fungsi predict digunakan untuk memprediksi data target berdasarkan data input. Fungsi akan melakukan forward propagation berdasarkan parameter bobot dan bias yang terbentuk (yang berasal dari tahapan ANN).

<a id='predict'></a>

In [10]:
def predict(X, params, activation_func):
    '''
    Input:
    params -- dict of params
    X -- input data
    
    Return:
    AL -- prediction
    '''
    X = X.T
    
    AL, cache = forward_propagation(X,params, activation_func)
    #predictions = (AL > 0.5)
    
    return np.squeeze(AL.T)#,predictions

<a id='plot'></a>

In [11]:
# Fungsi membuat plot / grafik -> agar support dark mode sih
def plot(x, y=None, label=None, title=None, 
         xlabel=None, ylabel=None, xticks=None, color='grey'):
    '''
    Plot x and y with pyplot
    Input:
    x -- data x di x axis, dapat > 1
    y -- data y di y axis (optional), dapat > 1
    label -- label data (khususnya jika x lebih dari 1)
    title -- judul grafik
    xlabel, ylabel -- label axis x dan y
    xticks -- list of x axis ticks
    color -- warna text grafik
    '''
    # init
    label2 = np.resize( np.array([None]),len(x)) if label is None else label
    plt.figure(figsize=(10,3))

    # plot per data
    if y is None:
        for i,lab in zip(x,label2): plt.plot(np.squeeze(i), label=lab)
    else:
        for i,j,lab in zip(x,y,label2): 
            plt.plot(np.squeeze(i),np.squeeze(j), label=lab)
    #plt.plot(x) if y is None else plt.plot(x,y)
    
    # label xy
    plt.ylabel(ylabel, color=color)
    plt.yticks(color=color)
    plt.xlabel(xlabel, color=color)
    if xticks is not None:
        plt.xticks([i for i in range(len(xticks))], xticks, color=color)
    else :
        plt.xticks(color=color)
    plt.title(title, color=color)
    
    # show
    if label is not None: plt.legend()
    plt.grid(True)
    plt.show()

## Fungsi pelengkap

In [ ]:
def conv_param(gen):
    '''
    Konversi nilai gen biar lebih manusiawi
    
    Input: 
    gen -- list nilai gen yg berupa bil
    
    Return:
    param -- dict berisi konversi nilai gen dari bil. ke ...tipenya (kali ya)
    '''
    param = {}
    
    # gen 1-4 -> hidden layer size n neuron size
    hidden_param = []
    for i in range(0,4): 
        # jika nilai gen tsb 0 ato negatif, maka berarti gada layer di urutan tsb dan selanjutnya
        if round(gen[i]) <= 0 : break
        # bulatin nilainya ke integer
        hidden_param.append(round(gen[i]))
    param['hidden'] = hidden_param
    
    # the rest gen konversi ke masing-masing tipe
    param['activation'] = ['identity','tanh', 'sigmoid','relu','leaky_relu'][int( np.floor(gen[4]) )]
    param['learning_rate'] = gen[5]
    param['momentum'] = gen[6]
    
    return param

def mape(y_true, y_pred): 
    '''Fungsi MAPE'''
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

## Implemen data dummy
Data dibawah merupakan hasil uji coba ANN menggunakan data dummy.

Perbandingan dengan sklearn